In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('../data/X_train.csv')

In [3]:
import sys
sys.path.append('../tools/')

In [4]:
from model_manager import model_manager as mm

In [5]:
svd = mm('svd_model', '../models/')
user_encoder = mm('user_encoder', '../models/')
item_encoder = mm('item_encoder', '../models/')
tv = mm('tv', '../models/')

In [6]:
models = {'tv': tv, 'user': user_encoder, 'item': item_encoder, 'svd': svd}

In [7]:
for _, model in models.items():
    model.load()

Done
Done
Done
Done


In [8]:
models = {name: model.model for name, model in models.items()}

In [9]:
train = train.sample(500000)

In [10]:
train['UserId'] = models['user'].transform(train['UserId'])
train['ProductId'] = models['item'].transform(train['ProductId'])

In [11]:
train['sentiment'] = train.sentiment.apply(eval)
train['sum_sentiment'] = train.sum_sentiment.apply(eval)

In [12]:
train['sentiment'] = train.sentiment.apply(np.array)
train['sum_sentiment'] = train.sum_sentiment.apply(np.array)

In [13]:
## user item text helpfulnum helpfulden senti sum_senti

In [14]:
m = np.concatenate([train[['HelpfulnessNumerator', 'HelpfulnessDenominator']].values, np.stack(train.values[:, -2]), np.stack(train.values[:, -1])],axis=1)

In [15]:
from text_preprocessor import preprocessor as pps

In [16]:
p = pps()

In [17]:
s = list(train[['Text']].values)

In [18]:
s = p.fit_transform(s)

100%|██████████| 500000/500000 [23:14<00:00, 358.57it/s]


In [19]:
from scipy import sparse

In [20]:
m = sparse.csr_matrix(m)

In [21]:
m = sparse.hstack([models['tv'].transform(s), m])

In [22]:
m = sparse.hstack([sparse.csr_matrix(models['svd'].pu[train.UserId.values]), sparse.csr_matrix(models['svd'].qi[train.ProductId.values]), m])

In [23]:
m.shape

(500000, 445)

In [24]:
np.savez('../data/train_matrix', X = m, y = train['Score'].values)